### Include study ID as as a feature and convert all features into categorial using OneHotEncoder

1. there is no signficant difference from the previous model which didnot include ID and all features used as numeric

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
combined = pd.read_csv("combined.csv")

In [4]:
combined.head()

,Internal Study ID,Race,Ethnicity,Sex,Currently taking AP courses,# of AP Courses,Currently part of extracurricular activities,Club hours,Academic hours,sports hours,...,Dinner_Red Meat Protein,Dinner_Other Protein,Dinner_Dairy,Dinner_Snacks,Additional_Fruits,Additional_Vegetables,Additional_Grains,Additional_Dairy,Additional_Snacks,Next_morning_mood
0,511,4,0,1,1,2,1,3,1,4,...,1,0,0,0,0,0,1,0,1,4
1,511,4,0,1,1,2,1,3,1,4,...,1,0,0,0,0,0,1,0,1,4
2,511,4,0,1,1,2,1,3,1,4,...,0,1,0,0,0,0,1,0,1,3
3,511,4,0,1,1,2,1,3,1,4,...,1,0,0,0,0,0,1,0,1,3
4,511,4,0,1,1,2,1,3,1,4,...,0,0,1,0,0,0,1,0,1,4


In [5]:
# separate label and features
from sklearn.model_selection import train_test_split

In [6]:
X = combined.drop(columns='Next_morning_mood')
y = combined["Next_morning_mood"]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
# treat all features as categorial, therefore no need to use scaler

scaler = StandardScaler()
X_train_prepared = scaler.fit_transform(X_train)
X_test_prepared = scaler.transform(X_test)

In [14]:
oneHot = OneHotEncoder(categories="auto")
X_train_prepared = oneHot.fit_transform(X_train)
X_test_prepared = oneHot.transform(X_test)

In [15]:
encoder = LabelEncoder()
y_train_prepared = encoder.fit_transform(y_train)
y_test_prepared = encoder.transform(y_test)

In [16]:
# Try svm model
from sklearn.svm import SVC
svm = SVC(kernel = "rbf", C=1, gamma=0.1)
svm.fit(X_train_prepared, y_train_prepared)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [17]:
print("Training accuracy is: ", svm.score(X_train_prepared, y_train_prepared))
print("Testing accuracy is: ", svm.score(X_test_prepared, y_test_prepared))

Training accuracy is:  0.9762470308788599
Testing accuracy is:  0.41509433962264153


In [18]:
# logistic model
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=100, penalty="l1")
logreg.fit(X_train_prepared, y_train_prepared)
print("Training accuracy is: ", logreg.score(X_train_prepared, y_train_prepared))
print("Testing accuracy is: ", logreg.score(X_test_prepared, y_test_prepared))

/Users/pengl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/pengl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Training accuracy is:  0.7624703087885986
Testing accuracy is:  0.41509433962264153


In [26]:
# random forest
from sklearn.ensemble.forest import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100, random_state = 46, max_features=7, max_depth=5)
forest.fit(X_train_prepared, y_train_prepared)
print("Training accuracy is: ", forest.score(X_train_prepared, y_train_prepared))
print("Testing accuracy is: ", forest.score(X_test_prepared, y_test_prepared))

Training accuracy is:  0.6175771971496437
Testing accuracy is:  0.4339622641509434


In [28]:
from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier(random_state = 0, max_depth=1, learning_rate=0.01, n_estimators=100)
gbrt.fit(X_train_prepared, y_train_prepared)
print("Training accuracy is: ", gbrt.score(X_train_prepared, y_train_prepared))
print("Testing accuracy is: ", gbrt.score(X_test_prepared, y_test_prepared))

Training accuracy is:  0.4821852731591449
Testing accuracy is:  0.39622641509433965


In [27]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(random_state = 46, max_depth = 5)
tree.fit(X_train_prepared, y_train_prepared)
print("Training accuracy is: ", tree.score(X_train_prepared, y_train_prepared))
print("Testing accuracy is: ", tree.score(X_test_prepared, y_test_prepared))

Training accuracy is:  0.5819477434679335
Testing accuracy is:  0.36792452830188677


In [ ]:
from sklearn.metrics import f1_score
models = [logreg, svm, tree, forest, gbrt]
model_names = ["logreg", "svm", "tree","forest", "gbrt"]
for i in range(len(models)):
    y_pred = models[i].predict(X_test_prepared)
    micro_f1 =f1_score(y_test_prepared, y_pred, average='micro')
    #macro_f1 =f1_score(y_test_prepared, y_pred, average='macro')
    print(model_names[i], "Micro-average F1 score: ", micro_f1)
    #print(model_names[i], "Micro-average F1 score: ", macro_f1)

### After doing onehot encoding for all features, it's not easy to show feature importance and draw decision tree

In [32]:
important_features = []
for label, feature in zip(list(X_train_prepared), forest.feature_importances_):  ## wrong
    print(label,": ", feature)

  (0, 15)	1.0
  (0, 25)	1.0
  (0, 30)	1.0
  (0, 33)	1.0
  (0, 34)	1.0
  (0, 37)	1.0
  (0, 39)	1.0
  (0, 41)	1.0
  (0, 46)	1.0
  (0, 49)	1.0
  (0, 50)	1.0
  (0, 54)	1.0
  (0, 55)	1.0
  (0, 60)	1.0
  (0, 61)	1.0
  (0, 66)	1.0
  (0, 72)	1.0
  (0, 79)	1.0
  (0, 85)	1.0
  (0, 88)	1.0
  (0, 93)	1.0
  (0, 98)	1.0
  (0, 100)	1.0
  (0, 105)	1.0
  (0, 108)	1.0
  (0, 112)	1.0
  (0, 115)	1.0
  (0, 118)	1.0
  (0, 119)	1.0
  (0, 121)	1.0
  (0, 124)	1.0
  (0, 125)	1.0
  (0, 128)	1.0
  (0, 129)	1.0
  (0, 132)	1.0
  (0, 134)	1.0
  (0, 136)	1.0
  (0, 137)	1.0
  (0, 140)	1.0
  (0, 141)	1.0
  (0, 144)	1.0
  (0, 145)	1.0
  (0, 148)	1.0
  (0, 149)	1.0
  (0, 151)	1.0
  (0, 154)	1.0
  (0, 155)	1.0
  (0, 158)	1.0 :  0.0013480059223233616
  (0, 1)	1.0
  (0, 26)	1.0
  (0, 30)	1.0
  (0, 33)	1.0
  (0, 34)	1.0
  (0, 36)	1.0
  (0, 39)	1.0
  (0, 41)	1.0
  (0, 45)	1.0
  (0, 49)	1.0
  (0, 50)	1.0
  (0, 53)	1.0
  (0, 55)	1.0
  (0, 59)	1.0
  (0, 61)	1.0
  (0, 65)	1.0
  (0, 74)	1.0
  (0, 80)	1.0
  (0, 85)	1.0
  (0, 90)	1.